<a href="https://colab.research.google.com/github/ARJUN108-verma/Vector-Database/blob/main/11_Using_BERT_Embedding_algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Using BERT Embedding Algorithm:-

 Indexing: Creating a Chroma Vectorstore

In [ ]:
%load_ext dotenv
%dotenv

In [ ]:
import pandas as pd

In [ ]:
files = pd.read_csv("/content/course_section_descriptions.csv", encoding='ANSI')

Data preprocessing

In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np

# Assuming 'files' DataFrame is already defined and includes necessary columns

# Define weights for different text components
weight_course_name = 5
weight_section_name = 3
weight_section_description = 2
weight_other = 1  # For other components like course technology and course description

# Create a unique identifier for each section combining course_id and section_id
files['unique_id'] = files['course_id'].astype(str) + '-' + files['section_id'].astype(str)

# Create metadata for each section
files['metadata'] = files.apply(lambda row: {
    "course_name": row['course_name'],
    "section_name": row['section_name'],
    "section_description": row['section_description']
}, axis=1)

In [ ]:
def create_embeddings(row):
    # Encode individual components
    emb_course_name = model.encode(row['course_name'], show_progress_bar=False) * weight_course_name
    emb_section_name = model.encode(row['section_name'], show_progress_bar=False) * weight_section_name
    emb_section_description = model.encode(row['section_description'], show_progress_bar=False) * weight_section_description
    emb_course_tech = model.encode(row['course_technology'], show_progress_bar=False) * weight_other
    emb_course_desc = model.encode(row['course_description'], show_progress_bar=False) * weight_other

    # Combine embeddings by averaging them
    combined_embedding = (emb_course_name + emb_section_name + emb_section_description + emb_course_tech + emb_course_desc) / (weight_course_name + weight_section_name + weight_section_description + 2 * weight_other)
    return combined_embedding

# Initialize the model
model = SentenceTransformer('multi-qa-distilbert-cos-v1')

# Apply the function to create weighted embeddings
files['embedding'] = files.apply(create_embeddings, axis=1)


Embedding :-

Connect to Pinecone Index

In [ ]:
import os
from pinecone import Pinecone, ServerlessSpec

In [ ]:
from dotenv import load_dotenv, find_dotenv

In [ ]:
load_dotenv(find_dotenv(), override = True)

In [ ]:
import pinecone
pc = Pinecone(api_key = os.environ.get("PINECONE_API_KEY"), environment = os.environ.get("PINECONE_ENV"))

In [ ]:
index = pc.Index("my-index")

In [ ]:
index.delete(delete_all = True)

In [ ]:
# Prepare the vectors for upserting
vectors_to_upsert = [(row['unique_id'], row['embedding'].tolist(), row['metadata']) for index, row in files.iterrows()]


In [ ]:

# Upsert data
index.upsert(vectors=vectors_to_upsert)

print("Data successfully upserted to Pinecone index.")

In [ ]:
# Ensure you've already initialized and configured Pinecone and the model
# If not, you need to run the initialization code provided earlier

# Create the query embedding
query = "lasso"
query_embedding = model.encode(query, show_progress_bar=False).tolist()

In [ ]:
query_results = index.query(
   # namespace="my-index",
    vector=[query_embedding],
    top_k=15,
    include_metadata=True
)

In [ ]:
import pprint

In [ ]:
score_threshold = 0.3

In [ ]:
# Assuming query_results are fetched and include metadata
for match in query_results['matches']:
    if match['score'] >= score_threshold:
        course_details = match.get('metadata', {})
        course_name = course_details.get('course_name', 'N/A')
        section_name = course_details.get('section_name', 'N/A')
        section_description = course_details.get('section_description', 'No description available')

        pprint.pprint(f"Matched item ID: {match['id']}, Score: {match['score']}")
        pprint.pprint(f"Course: {course_name}")
        pprint.pprint(f"Section: {section_name}, Description: {section_description}", width = 100)
       #pprint.pprint()